## What is CLIP ?


CLIP : Contrastive Language–Image Pretraining, it is a model that knows how to match between text and images , the role of clip is not to generate new conten but to match on a semantic metric the image and text we pass as input ,it is a zeroshot classifier it can be used for 
Classification
Semantic understanding ,
Labeling pictures
In the following notebook i will try to work on implementing CLIP "from scratch" to some extend but my interest is in understanding how the constrative pretraining work , i will try to implement the InfoNCE loss and answer questions that i thought of when working with this model


In [1]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTFeatureExtractor, AutoTokenizer, AutoModel
from PIL import Image
import torch.nn.functional as F
class ContrastiveLearningModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.device = "cpu"
        # image mdoel to get image embedding 
        self.imagemodel = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
        self.imagemodel.eval()

        # text model to get text embedding
        model_name = "bert-base-uncased"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.textmodel = AutoModel.from_pretrained(model_name)
        self.textmodel.eval()

        # Projection layers here the dimension size is 256
        projection_dim = 256
        self.image_proj = nn.Linear(self.imagemodel.config.hidden_size, projection_dim)# (B,256)
        self.text_proj = nn.Linear(self.textmodel.config.hidden_size, projection_dim) #(B,256)

    def forward(self, image, text,temp=0.07):
        # Process text
        text_inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            text_outputs = self.textmodel(**text_inputs)
        text_embedding = text_outputs.last_hidden_state[:, 0, :]  

        # Process image
        image_inputs = self.feature_extractor(images=image, return_tensors="pt")
        with torch.no_grad():
            image_outputs = self.imagemodel(**image_inputs)
        image_embedding = image_outputs.last_hidden_state[:, 0, :]  

        # Project to joint space
        image_proj = self.image_proj(image_embedding)  # shape: (1, projection_dim)  (batch size, projection dim)
        text_proj = self.text_proj(text_embedding)     # shape: (1, projection_dim)

        # Normalize
        image_proj = nn.functional.normalize(image_proj, dim=-1)
        text_proj = nn.functional.normalize(text_proj, dim=-1)
        #get simliairy
        logits=image_proj@text_proj.T  #(B,B)
        logits/=temp
        batch_size=image_proj.shape[0]
        labels=torch.arange(logits.shape[0], device=logits.device)

         #cross entropy

        loss_I = F.cross_entropy(logits.T, labels)
        loss_T = F.cross_entropy(logits, labels)
        loss = (loss_I + loss_T)/2.0 

        return loss,logits




2025-07-28 23:30:14.147211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753745414.414585      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753745414.495361      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# How the flow goes 

We get text and image embeddings from the text and image encoder (can be Vit or resnet)
We project both embeddings in the shared dimension space using Wi and Wt the matrixes of dimensions (B,dimension_size) both
Your raw features (img_feat, text_feat) are not guaranteed to have any alignment — even if ViT and BERT are pretrained!

We normalize both vectors
cause what we care about is what direction the embedding vecotr is going in , not how long ,big it is


Finally we Compute the similarity between both dimensions using dot product (textproj@imageproj) 
This simliairty matrix would be of dimension (B,B) the magic is that the datapoints on the diagonal would be the ones having the correct matching aka the highest simliarity score

During training

Measures cosine similarity of image-text pairs

Applies InfoNCE loss to push matching pairs together and others apart

Gradient updates adjust the projection weights to better align matches


# Why the data poitns on diagonal have same simliary score   
//imporve later

during the training , the model learns to push together models having same semantic meaning , pushing far away ones with differen semantic meaning , this is cause of the infonce loss

# what are we training here?

 what will be changing during training is the projection matrices wi and wt those learn neeed to learn to project the image and text of same semantic meaning in the same direction that way during matrix representation they both be on the diagnoanl?
 The weights of the projection layers learn how to map features into a space where semantic relationships are reflected geometrically (through vector direction and distance).

 The projection weights are trained (via InfoNCE loss) to rotate/scale/transform the original features in a way that:

dog image ↔ "a dog" → vectors point in the same direction

cat image ↔ "a cat" → same

dog image ↔ "a cat" → vectors point in different directions



resources 
https://medium.com/@paluchasz/understanding-openais-clip-model-6b52bade3fa3


# Implementation

In [2]:
# === Inference ===
clip = ContrastiveLearningModel()
clip.to(clip.device)
text = ["a cat"]  # list of text
image = Image.open('/kaggle/input/catttt/prettyboy.jpg').convert('RGB')  # single image

loss, logits = clip.forward(image, text)
print("Loss:", loss.item())


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loss: 0.0


# Impltted clip


In [3]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:

url = "/kaggle/input/catttt/prettyboy.jpg"
image = Image.open(url)


inputs = processor(text=["a photo of a lion", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities